In [0]:
from google.colab import drive

drive.mount('/content/drive')

In [0]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [0]:
f = open("/content/drive/My Drive/Malayalam_NLP/Data/pos_tagged_data/Malayalam_Tagged_Data.txt","r")
data = f.read()
print(len(data))
data = data.split(".        \RD_PUNC")
all_sent = []
for x in data:
    x = x.split("\n")
    v = []
    for y in x:
        if y != "":
            y = y.replace("\\","")
            y = y.split("        ")
            v.append( (y[0],y[1]) )
    v.append(('.','RD_PUNC'))
    all_sent.append(v)
del data
sentences = all_sent
del all_sent
print(len(sentences))

sentences = sentences[:5000]
print(len(sentences))


7041405
29077
5000


In [0]:
print(sentences[0])
other_words = []
for x in sentences:
  for y in x:
    if y[0] not in other_words:
      other_words.append(y[0])


[('മഹാത്മാഗാന്ധി', 'N_NNP'), ('രാഷ്ട്രപിതാവ്', 'N_NN'), ('ലക്ഷ്യങ്ങളെല്ലാം', 'N_NN'), ('കര്\u200d്മ്മഫലത്തിലെത്തിച്ച', 'V_VM_VNF'), ('മഹാൻ', 'N_NN'), ('.', 'RD_PUNC')]


In [0]:
print(len(other_words))

21879


In [0]:
sentence =[]
tags = []
count = 0
c = 0

for x in sentences:
  s = []
  t = []
  for y in x:
    if len(y)>=2:
      s.append(y[0])
      t.append(y[1])
      
  sentence.append((s,t))

  

In [0]:
print(sentence[0])

(['മഹാത്മാഗാന്ധി', 'രാഷ്ട്രപിതാവ്', 'ലക്ഷ്യങ്ങളെല്ലാം', 'കര്\u200d്മ്മഫലത്തിലെത്തിച്ച', 'മഹാൻ', '.'], ['N_NNP', 'N_NN', 'N_NN', 'V_VM_VNF', 'N_NN', 'RD_PUNC'])


In [0]:
print(len(sentence))
textData = sentence[2500:]
training_data = sentence

5000


In [0]:


def prepare_sequence(seq, to_ix):
    """Input: takes in a list of words, and a dictionary containing the index of the words
    Output: a tensor containing the indexes of the word"""
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)

In [0]:
word_to_ix = {} # This is the word dictionary which will contain the index to each word

for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix.keys():
            word_to_ix[word] = len(word_to_ix)
for word in other_words:
    if word not in word_to_ix.keys():
            word_to_ix[word] = len(word_to_ix)

# print(word_to_ix) # Just have a look at what it contains

tag_to_ix = {'N_NNP': 0, 'N_NN': 1, 'V_VM_VNF': 2, 'RD_PUNC': 3, 'QT_QTF': 4, 'RB': 5, 'PR_PRF': 6, 'JJ': 7, 'PR_PRP': 8, 'V_VAUX': 9, 'QT_QTO': 10, 'RP_INTF': 11, 'RD_RDF': 12, 'DM_DMD': 13, 'DM_DMR': 14, 'CC_CCD': 15, 'PSP': 16, 'V_VM_VF': 17, 'N_NST': 18, 'CC_CCS': 19, 'RP_NEG': 20, 'V_VM_VINF': 21, 'DM_DMQ': 22, 'PR_PRL': 23, 'V_VM': 24, 'RP_RPD': 25, 'PR_PRC': 26, 'V_VN': 27, 'RP_INJ': 28, 'QT_QTC': 29, 'PR_PRQ': 30,'CC_CCS_UT': 31,'RD_UNK':32}
   #{"DET": 0, "NN": 1, "V": 2, "ADJ": 3, "ADV": 4, "PRP": 5, "PRN": 6} # This dictionary contains the indices of the tags

EMBEDDING_DIM = 64
HIDDEN_DIM = 64

In [0]:
class LSTMTagger(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size, target_size):
        super(LSTMTagger, self).__init__()
        
        self.hidden_dim = hidden_dim
        
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        self.hidden2tag = nn.Linear(hidden_dim, target_size)
        
    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [12]:
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix.keys()), len(tag_to_ix.keys()))

# Define the loss function as the Negative Log Likelihood loss (NLLLoss)
loss_function = nn.NLLLoss()

# We will be using a simple SGD optimizer
optimizer = optim.SGD(model.parameters(), lr=0.1)

# The test sentence

    
print("Training Started")
for epoch in range(300):
    print("epoch :", epoch)
    for sentence, tags in training_data[:2500]:
        model.zero_grad()
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)
        
        tag_scores = model(sentence_in)
        
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()
print("Training Finished!!!\nAgain testing on unknown data")
with torch.no_grad():
    for seq in [seq1, seq2]:
        inputs = prepare_sequence(seq, word_to_ix)
        tag_scores = model(inputs)
        _, indices = torch.max(tag_scores, 1)
        ret = []
        for i in range(len(indices)):
            for key, value in tag_to_ix.items():
                if indices[i] == value:
                    ret.append((seq[i], key))
        print(ret)

Training Started
epoch : 0
epoch : 1
epoch : 2
epoch : 3
epoch : 4
epoch : 5
epoch : 6
epoch : 7
epoch : 8
epoch : 9
epoch : 10
epoch : 11
epoch : 12
epoch : 13
epoch : 14
epoch : 15
epoch : 16
epoch : 17
epoch : 18
epoch : 19
epoch : 20
epoch : 21
epoch : 22
epoch : 23
epoch : 24
epoch : 25
epoch : 26
epoch : 27
epoch : 28
epoch : 29
epoch : 30
epoch : 31
epoch : 32
epoch : 33
epoch : 34
epoch : 35
epoch : 36
epoch : 37
epoch : 38
epoch : 39
epoch : 40
epoch : 41
epoch : 42
epoch : 43
epoch : 44
epoch : 45
epoch : 46
epoch : 47
epoch : 48
epoch : 49
epoch : 50
epoch : 51
epoch : 52
epoch : 53
epoch : 54
epoch : 55
epoch : 56
epoch : 57
epoch : 58
epoch : 59
epoch : 60
epoch : 61
epoch : 62
epoch : 63
epoch : 64
epoch : 65
epoch : 66
epoch : 67
epoch : 68
epoch : 69
epoch : 70
epoch : 71
epoch : 72
epoch : 73
epoch : 74
epoch : 75
epoch : 76
epoch : 77
epoch : 78
epoch : 79
epoch : 80
epoch : 81
epoch : 82
epoch : 83
epoch : 84
epoch : 85
epoch : 86
epoch : 87
epoch : 88
epoch : 89
epo

NameError: ignored

In [0]:
print(len(training_data))

2500


In [0]:
torch.save(model.state_dict(),'/content/drive/My Drive/NLP Research/s2.model')

In [0]:
#seq1 = 'മഹാത്മാഗാന്ധി  രാഷ്ട്രപിതാവ് ലക്ഷ്യങ്ങളെല്ലാം കര്\u200d്മ്മഫലത്തിലെത്തിച്ച മഹാൻ .'.split()
with torch.no_grad():
    for seq in textData:
        inputs = prepare_sequence(seq, word_to_ix)
        tag_scores = model(inputs)
        _, indices = torch.max(tag_scores, 1)
        ret = []
        for i in range(len(indices)):
            for key, value in tag_to_ix.items():
                if indices[i] == value:
                    ret.append((seq[i], key))
        print(ret)

TypeError: ignored

In [0]:
sentence2 =[]
tags = []
count = 0
c = 0

for x in sentences[2500:]:
  s = []
  t = []
  for y in x:
    if len(y)>=2:
      s.append(y[0])
      t.append(y[1])
      
  sentence2.append((s,t))

In [0]:
#print(sentence2[0])
testData2 = []
for x in testData[:500]:
  sub = ""
  for y in x[0]:
    sub+=y
    sub+=" "
  testData2.append(sub)

In [0]:
print(testData[1])

IndexError: ignored

In [0]:
seq1 =['അവര്‍ ബക്കററ് പലതവണ താഴേക്കിറക്കിയതാണു് .'.split()] 
#seq1 = []

with torch.no_grad():
      for seq in seq1:

          inputs = prepare_sequence(seq, word_to_ix)
          tag_scores = model(inputs)
          _, indices = torch.max(tag_scores, 1)
          ret = []
          for i in range(len(indices)):
              for key, value in tag_to_ix.items():
                  if indices[i] == value:
                      ret.append((seq[i], key))
          print(ret)
  

NameError: ignored